In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [8]:
prompt = f""" 
Te pongo al día. Estamos desarrollando una base de datos para una app de reservas de restaurantes.
Tu me estás ayudando. La base de datos será escrita en postgresql.

Dada la siguiente tabla:
CREATE TABLE reservations (
  date DATE NOT NULL DEFAULT CURRENT_DATE,
  time TIME_OPTIONS_ENUM NOT NULL,
  res_number INTEGER NOT NULL DEFAULT 0, -- reservation number, if there isn't one, use 0
  name VARCHAR(100),
  room ROOM_OPTIONS_ENUM, -- if res number is 0, room must be S/N
  is_bonus BOOLEAN DEFAULT FALSE, -- if true, means cost is 0 and bonus_qty is > 0 Meal plan is AI
  bonus_qty INTEGER DEFAULT 0,
  meal_plan MEAL_PLAN_ENUM, -- if is_bonus is true, meal_plan must be AI
  pax_number INTEGER,
  cost NUMERIC(10,2), -- if is_bonus is true, cost must be 0
  observations TEXT,
  is_noshow BOOLEAN DEFAULT FALSE,
  created_at TIMESTAMP DEFAULT NOW(),
  updated_at TIMESTAMP DEFAULT NOW(),
  is_deleted BOOLEAN DEFAULT FALSE,
  PRIMARY KEY (date, time, res_number, name, room)
);

Quiero que me ayudes a escribir un procedimiento almacenado que tenga por parámetro de entra un número de reserva, y me devuelva todos los registros de esa reserva que tengan is_bous igual a true.

Quiero el output en PLPGSQL.
"""
print(get_completion(prompt))

Para escribir el procedimiento almacenado en PLPGSQL que cumpla con los requisitos mencionados, puedes utilizar el siguiente código:

CREATE OR REPLACE FUNCTION get_bonus_reservations(reservation_number INTEGER)
RETURNS TABLE (
  date DATE,
  time TIME_OPTIONS_ENUM,
  res_number INTEGER,
  name VARCHAR(100),
  room ROOM_OPTIONS_ENUM,
  is_bonus BOOLEAN,
  bonus_qty INTEGER,
  meal_plan MEAL_PLAN_ENUM,
  pax_number INTEGER,
  cost NUMERIC(10,2),
  observations TEXT,
  is_noshow BOOLEAN,
  created_at TIMESTAMP,
  updated_at TIMESTAMP,
  is_deleted BOOLEAN
) AS $$
BEGIN
  RETURN QUERY SELECT * FROM reservations WHERE res_number = reservation_number AND is_bonus = TRUE;
END;
$$ LANGUAGE plpgsql;

Este procedimiento almacenado recibe como parámetro de entrada el número de reserva que se desea buscar y devuelve una tabla con todos los registros de esa reserva que tengan is_bonus igual a true. Para ello, se utiliza la cláusula RETURN QUERY para realizar la consulta a la tabla reservations y f

In [9]:
prompt = f"""
Te muestro las siguientes 2 tablas.
DROP TABLE IF EXISTS agenda;

CREATE TABLE agenda (
  fecha DATE NOT NULL DEFAULT CURRENT_DATE PRIMARY KEY,
  restaurant_theme_id INTEGER NOT NULL,
  t1900 INTEGER NOT NULL DEFAULT 10,
  t1915 INTEGER NOT NULL DEFAULT 8,
  t1930 INTEGER NOT NULL DEFAULT 6,
  t1945 INTEGER NOT NULL DEFAULT 0,
  t2000 INTEGER NOT NULL DEFAULT 4,
  t2015 INTEGER NOT NULL DEFAULT 6,
  t2030 INTEGER NOT NULL DEFAULT 4,
  t2045 INTEGER NOT NULL DEFAULT 6,
  t2100 INTEGER NOT NULL DEFAULT 6,
  t2115 INTEGER NOT NULL DEFAULT 6,
  t2130 INTEGER NOT NULL DEFAULT 4,
  t2145 INTEGER NOT NULL DEFAULT 4,
  created_at TIMESTAMP DEFAULT NOW(),
  updated_at TIMESTAMP DEFAULT NOW(),
  is_deleted BOOLEAN DEFAULT FALSE
);

CREATE TABLE reservations (
  fecha DATE NOT NULL DEFAULT CURRENT_DATE,
  hora TIME_OPTIONS_ENUM NOT NULL,
  res_number INTEGER NOT NULL DEFAULT 0, -- reservation number, if there isn't one, use 0
  res_name VARCHAR(100),
  room ROOM_OPTIONS_ENUM, -- if res number is 0, room must be S/N
  is_bonus BOOLEAN DEFAULT FALSE, -- if true, means cost is 0 and bonus_qty is > 0 Meal plan is AI
  bonus_qty INTEGER DEFAULT 0,
  meal_plan MEAL_PLAN_ENUM, -- if is_bonus is true, meal_plan must be AI
  pax_number INTEGER,
  cost NUMERIC(10,2), -- if is_bonus is true, cost must be 0
  observations TEXT,
  is_noshow BOOLEAN DEFAULT FALSE,
  created_at TIMESTAMP DEFAULT NOW(),
  updated_at TIMESTAMP DEFAULT NOW(),
  is_deleted BOOLEAN DEFAULT FALSE,
  PRIMARY KEY (fecha, hora, res_number, res_name, room)
);

La columna t1900 de agenda representa el máximo número de personas que pueden reservar a las 19:00 en la fecha indicada por la columna fecha de agenda.
La columna pax_number de reservations representa el número de personas que reservan.
La columna hora representa la hora de la reserva en reservations.
La columna fecha en reservations representa la fecha de la reserva.

Quiero que me ayudes a escribir un procedimiento almacenado
Debe tener por parámetro de entrada una fecha y una hora, y me devuelva el número de personas que pueden reservar en esa fecha y hora.
"""
print(get_completion(prompt))

CREATE OR REPLACE FUNCTION get_available_seats(fecha DATE, hora TIME_OPTIONS_ENUM) 
RETURNS INTEGER AS $$
DECLARE
  max_seats INTEGER;
  reserved_seats INTEGER;
BEGIN
  SELECT t1900 INTO max_seats FROM agenda WHERE fecha = fecha;
  SELECT SUM(pax_number) INTO reserved_seats FROM reservations WHERE fecha = fecha AND hora = hora;
  IF max_seats IS NULL THEN
    RETURN -1; -- fecha no encontrada en agenda
  ELSIF reserved_seats IS NULL THEN
    RETURN max_seats; -- no hay reservas para esa fecha y hora
  ELSE
    RETURN max_seats - reserved_seats; -- devuelve el número de asientos disponibles
  END IF;
END;
$$ LANGUAGE plpgsql;
